在实际生产环境中，对代码进行调试和性能分析，是一个永远都逃不开的话题。调试和性能分析的主要场景，通常有这么三个： 
1. 一是代码本身有问题，需要我们找到 root cause 并修复； 
2. 二是代码效率有问题，比如过度浪费资源，增加 latency，因此需要我们 debug； 
3. 三是在开发新的 feature 时，一般都需要测试。

# 用 pdb 进行代码调试

## pdb 的必要性

1. `print()`适合小一些的程序。
2. IDE内置debug工具，设置断点，针对单一语言较好，对于混合代码不支持UI形式的断点调试功能，或者针对某个功能模块支持。另外，考虑到不少代码已经挪到了类似 Jupyter 的 Notebook 中，往往就要求开发者使用命令行的形式，来对代码进行调试。
3. Python的`pdb`，是其自带的一个调试库。为Python提供交互式的源代码调试功能，是命令行版本的IDE断点调试器。


## 如何使用 pdb

首先需要启动pdb，加入“import pdb”和“pdb.set_trace()”这两行代码就行了。

In [ ]:
import pdb

a = 1
b = 2
pdb.set_trace() # 设置断点
c = 3
print(a + b + c)

当我们运行这个程序时时，它的输出界面`e:\git\learning\python\test.py(66)<module>()`，表示程序已经运行到了“pdb.set_trace()”这行，并且暂停了下来，等待用户输入。

这时，我们就可以执行，在 IDE 断点调试器中可以执行的一切操作，比如**打印，语法是`p <expression>`**：
```python
(Pdb) p a
1
(Pdb) p b
2
(Pdb) p c
*** NameError: name 'c' is not defined
```

除了打印，常见的操作还有`n`，表示继续执行代码到下一行，用法如下：
```python
(Pdb) n
-> print(a + b + c)
```

命令`l`，则表示列举出当前代码行上下的 11 行源代码，方便开发者熟悉当前断点周围的代码状态：
```python
(Pdb) l
 62
 63     a = 1
 64     b = 2
 65     pdb.set_trace()
 66     c = 3
 67  -> print(a + b + c)
[EOF]
```

命令`s`，就是 step into 的意思，即进入相对应的代码内部。这时，命令行中会显示”--Call--“的字样，当你执行完内部的代码块后，命令行中则会出现”--Return--“的字样。
```python
6
--Return--
> e:\git\learning\python\test.py(67)<module>()->None
-> print(a + b + c)
```

 ### pdb 基本命令
 
 | 命令 |	解释 |
 |-----|--------|
|break 或 b|	设置断点|
|continue 或 c|	继续执行程序|
|list 或 l|	查看当前行的代码段|
|step 或 s|	进入函数（进入 for 循环用 next 而不是用 step）|
|return 或 r|	执行代码直到从当前函数返回|
|next 或 n|	执行下一行|
|up 或 u|	返回到上个调用点（不是上一行）|
|p x|	打印变量x的值|
|exit 或 q|	中止调试，退出程序|
|help|	帮助|


1. 与之相对应的命令”r“，表示 step out，即继续执行，直到当前的函数完成返回。 
2. 命令”b [ ([filename:]lineno | function) [, condition] ]“可以用来设置断点。比方说，我想要在代码中的第 10 行，再加一个断点，那么在 pdb 模式下输入”b 11“即可。 
3. 而”c“则表示一直执行程序，直到遇到下一个断点。

In [ ]:
def func():
    print('enter func()') 
a = 1
b = 2 
import pdb 
pdb.set_trace() 
func() 
c = 3 
print(a + b + c)


```pdb
> e:\git\learning\python\test.py(79)<module>()
-> func()
(Pdb) s        #使用命令”s“进入了函数 func() 的内部，显示”--Call--“
--Call--
> e:\git\learning\python\test.py(73)func()
-> def func():
(Pdb) l
 68     # c = 3
 69     # print(a + b + c)
 70  
 71  
 72  
 73  -> def func():
 74         print('enter func()')
 75     a = 1
 76     b = 2
 77     import pdb
 78     pdb.set_trace()
(Pdb) n
> e:\git\learning\python\test.py(74)func()
-> print('enter func()')
(Pdb) n        # 当我们执行完函数 func() 内部语句并跳出后，显示”--Return--“
enter func()
--Return--
> e:\git\learning\python\test.py(74)func()->None
-> print('enter func()')
(Pdb) n
> e:\git\learning\python\test.py(80)<module>()
-> c = 3
```

# 用 cProfile 进行性能分析

除了要对程序进行调试，性能分析也是每个开发者的必备技能。日常工作中，我们常常会遇到这样的问题：在线上，我发现产品的某个功能模块效率低下，延迟（latency）高，占用的资源多，但却不知道是哪里出了问题。 这时，对代码进行 profile 就显得异常重要了。

这里所谓的 profile，是指对代码的每个部分进行动态的分析，比如准确计算出每个模块消耗的时间等。这样你就可以知道程序的瓶颈所在，从而对其进行修正或优化。当然，这并不需要你花费特别大的力气，在 Python 中，这些需求用 cProfile 就可以实现。

比如我想计算斐波拉契数列，运用递归思想，我们很容易就能写出下面这样的代码：

In [10]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    res = []
    if n > 0:
        res.extend(fib_seq(n-1))
    res.append(fib(n))
    return res

fib_seq(30)


[0,
 1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765,
 10946,
 17711,
 28657,
 46368,
 75025,
 121393,
 196418,
 317811,
 514229,
 832040]

接下来，我想要测试一下这段代码总的效率以及各个部分的效率。那么，我就只需在开头导入 cProfile 这个模块，并且在最后运行 cProfile.run() 就可以了：

In [11]:
import cProfile
# def fib(n)
# def fib_seq(n):
cProfile.run('fib_seq(30)')

         7049218 function calls (96 primitive calls) in 2.552 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
7049123/31    2.552    0.000    2.552    0.082 2976067564.py:1(fib)
     31/1    0.000    0.000    2.552    2.552 2976067564.py:9(fib_seq)
        1    0.000    0.000    2.552    2.552 <string>:1(<module>)
        1    0.000    0.000    2.552    2.552 {built-in method builtins.exec}
       31    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       30    0.000    0.000    0.000    0.000 {method 'extend' of 'list' objects}




或者更简单一些，直接在运行脚本的命令中，加入选项“-m cProfile”也很方便：`python3 -m cProfile xxx.py`

运行完毕后，我们可以看到下面这个输出界面：

```
         7049218 function calls (96 primitive calls) in 2.552 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
7049123/31    2.552    0.000    2.552    0.082 2976067564.py:1(fib)
     31/1    0.000    0.000    2.552    2.552 2976067564.py:9(fib_seq)
        1    0.000    0.000    2.552    2.552 <string>:1(<module>)
        1    0.000    0.000    2.552    2.552 {built-in method builtins.exec}
       31    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       30    0.000    0.000    0.000    0.000 {method 'extend' of 'list' objects}
```

参数定义如下：
1. ncalls，是指相应代码 / 函数被调用的次数； 
2. tottime，是指对应代码 / 函数总共执行所需要的时间（注意，并不包括它调用的其他代码 / 函数的执行时间）； 
3. percall，就是上述两者相除的结果，也就是 tottime / ncalls； 
4. cumtime，则是指对应代码 / 函数总共执行所需要的时间，这里包括了它调用的其他代码 / 函数的执行时间； 
5. percall，则是 cumtime 和 ncalls 相除的平均结果。

这段程序执行效率的瓶颈，在于第二行的函数 fib()，它被调用了 700 多万次。

有没有什么办法可以提高改进呢？答案是肯定的。通过观察，我们发现，程序中有很多对 fib() 的调用，其实是重复的，那我们就可以用字典来保存计算过的结果，防止重复。改进后的代码如下所示：

In [12]:
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

@memoize
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)


def fib_seq(n):
    res = []
    if n > 0:
        res.extend(fib_seq(n-1))
    res.append(fib(n))
    return res

fib_seq(30)


[0,
 1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765,
 10946,
 17711,
 28657,
 46368,
 75025,
 121393,
 196418,
 317811,
 514229,
 832040]

In [13]:
import cProfile
# def fib(n)
# def fib_seq(n):
cProfile.run('fib_seq(30)')

         126 function calls (96 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     31/1    0.000    0.000    0.000    0.000 2888787982.py:19(fib_seq)
       31    0.000    0.000    0.000    0.000 2888787982.py:3(helper)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
       31    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       30    0.000    0.000    0.000    0.000 {method 'extend' of 'list' objects}


